In [226]:
#Import packages
import pandas as pd
import numpy as np

# Read in data. Remove commas, drop out failed-pricing loans, convert amount to integers
loans=pd.read_csv('BestEx_fixed.csv')
loans['Amount']=loans['Amount'].str.replace(',','')
loans=loans.dropna(axis=0,how='any',subset=['Amount']);
loans['Amount']=loans['Amount'].astype(int);
temp=loans

# Find and replace various valmethodid's into particular groups. 
l=loans.ValMethodID.unique()
Wells=[]
Fannie_Spec=[]
Fannie_Cash=[]
Freddie_Spec=[]
Freddie_Cash=[]
Chase=[]
MBS=[]
for x in l:
    if 'MBS' in x:
        MBS.append(x)
    else:
        if 'Wells' in x:
            Wells.append(x)
        elif 'Fannie_Cash'==x:
            Fannie_Cash.append(x)
        elif 'Freddie_Cash'==x:
            Freddie_Cash.append(x)
        elif 'Fannie' in x:
            Fannie_Spec.append(x)
        elif 'Freddie' in x:
            Freddie_Spec.append(x)
        elif 'Chase' in x:
            Chase.append(x)
        else:
            Bulk.append(x)
            
loans=loans.replace(Wells, 'Wells').replace(Fannie_Spec,'Fannie_Spec').replace(Freddie_Spec,'Freddie_Spec').replace(MBS,'MBS').replace(Bulk,'Bulk')

#Determine the margin at which the winning investor beat other competitors
Rank1_Margin=[]
for r in range (0,loans.shape[0]):
    x=loans.loc[r].loc['Rank 1 Price']
    second=0
    for i in list(loans.loc[r][30:45]):
        if i>second:
            if i<x:
                second=i
    if second==0:
        Rank1_Margin.append(0)
    else:
        Rank1_Margin.append(100*(x-second))

new=pd.Series(Rank1_Margin);
loans['Rank1_Margin']=new.values

#Pivot data. Group by valmethod and sum loan amounts. In addition give a count.
df1=loans.groupby(['ValMethodID'])[['Amount']].sum();
df2=loans.groupby(['ValMethodID'])[['ProcID']].count();
df3=loans.groupby(['ValMethodID'])[['Rank1_Margin']].mean()
df=df1.join(df2.join(df3,how='left'), how='left')
print(df)

#Statistics for Wells
loans_volume=df['Amount'].sum()
loans_count=df['ProcID'].sum()
n1=df.iloc[6,0]
n2=df.iloc[6,1]
print('Percentage of volume Wells won', n1/loans_volume)
print('Percentage of loans Wells won', n2/loans_count)
print('Of loans that Wells won, Wells won by', df.iloc[6,2], 'bps')

                Amount  ProcID  Rank1_Margin
ValMethodID                                 
Bulk          11637751      36     33.548889
Fannie_Cash    1248000       5      1.680000
Fannie_Spec    5028960      36     13.992778
Freddie_Cash   2466350      10     11.956000
Freddie_Spec   5904235      26     11.750000
MBS            3688117      15     14.634000
Wells          4808318      16     25.506250
Percentage of volume Wells won 0.13824263088
Percentage of loans Wells won 0.111111111111
Of loans that Wells won, Wells won by 25.50625 bps
